In [1]:
import numpy as np
import pandas as pd
import spacy
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import GradientBoostingRegressor


# Create the dataframes

In [2]:
df_train = pd.read_csv('home_depot_data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('home_depot_data/test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('home_depot_data/attributes.csv')
df_pro_desc = pd.read_csv('home_depot_data/product_descriptions.csv')

# Add stemmer

In [3]:
stemmer = SnowballStemmer('english')
def str_stemmer(s):
	return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())

In [4]:
df_train

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67
...,...,...,...,...,...
74062,221457,206638,Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...,tv riser glass,1.00
74063,221458,206639,Philips 40-Watt Halogen R20 Flood Light Bulb (...,r20 halogen light,3.00
74064,221463,206641,Schlage Camelot In-Active Aged Bronze Handlese...,schlage lock siena half dummy knob with,2.33
74065,221471,206648,Plastec 11 in. x 24 in. Rose Garden Wall Decor...,zen garden decor,3.00


# Dataframe processing

In [5]:
df_train_base = df_train.drop(['search_term','product_title'],axis=1)
df_train['search_term'] = df_train['search_term'].map(lambda x:str_stemmer(x))
df_train['product_title'] = df_train['product_title'].map(lambda x:str_stemmer(x))
df_train['len_of_query'] = df_train['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_train['product_info'] = df_train['search_term']+"\t"+df_train['product_title']
df_train['word_in_title'] = df_train['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_complete=df_train
df_train = df_train.drop(['search_term','product_title', 'product_info'],axis=1)

In [6]:
train, test = train_test_split(df_train, test_size=0.2, random_state=42)
base_train, base_test = train_test_split(df_train_base, test_size=0.2, random_state=42)

# Define test and training sets

In [8]:
id_test = test['id']
y_train = train['relevance'].values
X_train = train.drop(['id','relevance'],axis=1).values
X_test = test.drop(['id','relevance'],axis=1).values
y_test = test['relevance'].values

In [9]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=42)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [10]:
id_base_test = base_test['id']
y_base_train = base_train['relevance'].values
X_base_train = base_train.drop(['id','relevance'],axis=1).values
X_base_test = base_test.drop(['id','relevance'],axis=1).values
y_base_test = base_test['relevance'].values

In [11]:
rf_base = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=42)
clf_base = BaggingRegressor(rf_base, n_estimators=45, max_samples=0.1, random_state=42)
clf_base.fit(X_base_train, y_base_train)
y_base_pred = clf_base.predict(X_base_test)

In [12]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_base = np.sqrt(mean_squared_error(y_base_test, y_base_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_base:.4f}")


Root Mean Squared Error (RMSE): 0.4842
Root Mean Squared Error (RMSE): 0.5245


# Improving the matching

In [13]:
# Group by 'name' and count number of appearances
attribute_counts = df_attr['name'].value_counts().reset_index()
attribute_counts.columns = ['attribute_name', 'count']

# Show top attributes
print(attribute_counts.head(20))


                 attribute_name  count
0                MFG Brand Name  86250
1                      Bullet02  86248
2                      Bullet03  86226
3                      Bullet04  86174
4                      Bullet01  85940
5           Product Width (in.)  61137
6                      Bullet05  60529
7          Product Height (in.)  54698
8           Product Depth (in.)  53652
9          Product Weight (lb.)  45175
10                     Bullet06  44901
11                 Color Family  41508
12                     Bullet07  34349
13                     Material  31500
14                 Color/Finish  28564
15                     Bullet08  26645
16  Certifications and Listings  24583
17                     Bullet09  20567
18       Assembled Height (in.)  18299
19        Assembled Width (in.)  18263


In [14]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_md")

# Clean attribute data
df_attr['name'] = df_attr['name'].fillna('').str.lower()
df_attr['value'] = df_attr['value'].astype(str).str.lower()

# ----- 1. Combine bullet values -----
df_bullets = df_attr[df_attr['name'].str.startswith("bullet")].copy()
bullet_texts = df_bullets.groupby('product_uid')['value'].apply(lambda x: ' '.join(x)).reset_index()
bullet_texts = bullet_texts.rename(columns={'value': 'all_bullets'})

# Merge bullets into main dataframe
df_complete = df_complete.merge(bullet_texts, on='product_uid', how='left')

# ----- 2. Define attributes to include -----
selected_attrs = ['mfg brand name', 'material', 'color/finish', 'certifications and listings']
attributes = selected_attrs + ['all_bullets']

# ----- 3. Loop and compute SpaCy similarity -----
for attr_name in attributes:
    print(f"Processing: {attr_name}")

    if attr_name == 'all_bullets':
        attr_col = 'all_bullets'
    else:
        attr_subset = df_attr[df_attr['name'] == attr_name].drop_duplicates('product_uid')
        attr_subset = attr_subset.rename(columns={'value': f'{attr_name}_value'})
        df_complete = df_complete.merge(attr_subset[['product_uid', f'{attr_name}_value']], on='product_uid', how='left')
        attr_col = f'{attr_name}_value'

    def safe_similarity(row):
        if pd.isna(row['search_term']) or pd.isna(row[attr_col]):
            return 0.0
        doc_query = nlp(row['search_term'])
        doc_value = nlp(row[attr_col])
        if doc_query.vector_norm == 0 or doc_value.vector_norm == 0:
            return 0.0
        return doc_query.similarity(doc_value)

    df_complete[f'spacy_sim_{attr_name}'] = df_complete.apply(safe_similarity, axis=1)

Processing: mfg brand name
Processing: material
Processing: color/finish
Processing: certifications and listings
Processing: all_bullets


In [15]:
# Fit on combined corpus
corpus = pd.concat([df_complete['search_term'], df_complete['product_title']], axis=0)
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

# Transform search terms and titles separately
tfidf_query = vectorizer.transform(df_complete['search_term'])
tfidf_title = vectorizer.transform(df_complete['product_title'])

# Compute cosine similarity for each row
df_complete['cosine_title'] = [
    cosine_similarity(tfidf_query[i], tfidf_title[i])[0][0]
    for i in range(len(df_complete))
]

# Add description
df_complete = df_complete.merge(df_pro_desc, on='product_uid', how='left')
df_complete['product_description'] = df_complete['product_description'].fillna("")

# TF-IDF on search_term and product_description
corpus = pd.concat([df_complete['search_term'], df_complete['product_description']], axis=0)
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

tfidf_query = vectorizer.transform(df_complete['search_term'])
tfidf_desc = vectorizer.transform(df_complete['product_description'])

df_complete['cosine_description'] = [
    cosine_similarity(tfidf_query[i], tfidf_desc[i])[0][0]
    for i in range(len(df_complete))
]


In [16]:
def spacy_similarity(row):
    doc1 = nlp(row['search_term'])
    doc2 = nlp(row['product_title'])
    if doc1.vector_norm == 0 or doc2.vector_norm == 0:
        return 0.0
    return doc1.similarity(doc2)

df_complete['spacy_sim_title'] = df_complete.apply(spacy_similarity, axis=1)


In [17]:
df_complete

,id,product_uid,product_title,search_term,relevance,len_of_query,product_info,word_in_title,all_bullets,mfg brand name_value,...,spacy_sim_material,color/finish_value,spacy_sim_color/finish,certifications and listings_value,spacy_sim_certifications and listings,spacy_sim_all_bullets,cosine_title,product_description,cosine_description,spacy_sim_title
0,2,100001,simpson strong-ti 12-gaug angl,angl bracket,3.00,2,angl bracket\tsimpson strong-ti 12-gaug angl,1,versatile connector for various 90° connection...,simpson strong-tie,...,0.298688,NaN,0.000000,NaN,0.000000,0.680476,0.304570,"Not only do angles make joints stronger, they ...",0.000000,0.214588
1,3,100001,simpson strong-ti 12-gaug angl,l bracket,2.50,2,l bracket\tsimpson strong-ti 12-gaug angl,1,versatile connector for various 90° connection...,simpson strong-tie,...,0.186022,NaN,0.000000,NaN,0.000000,0.536840,0.000000,"Not only do angles make joints stronger, they ...",0.000000,0.418137
2,9,100002,behr premium textur deckov 1-gal. #sc-141 tugb...,deck over,3.00,2,deck over\tbehr premium textur deckov 1-gal. #...,1,"revives wood and composite decks, railings, po...",behr premium textured deckover,...,0.000000,tugboat,-0.125623,NaN,0.000000,0.527105,0.000000,BEHR Premium Textured DECKOVER is an innovativ...,0.052187,0.345363
3,16,100005,delta vero 1-handl shower onli faucet trim kit...,rain shower head,2.33,3,rain shower head\tdelta vero 1-handl shower on...,1,"includes the trim kit only, the rough-in kit (...",delta,...,0.000000,chrome,0.146512,"ada compliant,csa certified,iapmo certified",0.392925,0.508179,0.105734,Update your bathroom with the Delta Vero Singl...,0.041532,0.469092
4,17,100005,delta vero 1-handl shower onli faucet trim kit...,shower onli faucet,2.67,3,shower onli faucet\tdelta vero 1-handl shower ...,3,"includes the trim kit only, the rough-in kit (...",delta,...,0.000000,chrome,0.207017,"ada compliant,csa certified,iapmo certified",0.248380,0.250932,0.457361,Update your bathroom with the Delta Vero Singl...,0.064685,0.620041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74062,221457,206638,atlant windowpan 576 cd or 192 dvd blu-ray or ...,tv riser glass,1.00,3,tv riser glass\tatlant windowpan 576 cd or 192...,1,"holds 216 blu-rays, 192 dvds or 576 cds temper...",atlantic,...,0.440012,maple,0.285542,NaN,0.000000,0.458558,0.059904,"Atlantic, Inc. 94835722 Uniquely designed for ...",0.052217,0.521980
74063,221458,206639,philip 40-watt halogen r20 flood light bulb (1...,r20 halogen light,3.00,3,r20 halogen light\tphilip 40-watt halogen r20 ...,3,brightness: 590 lumens estimated yearly energy...,philips,...,0.000000,NaN,0.000000,NaN,0.000000,0.401885,0.643635,Philips Energy Advantage lamps use less energy...,0.140583,0.595860
74064,221463,206641,schlage camelot in-act age bronz handleset wit...,schlage lock siena half dummi knob with,2.33,7,schlage lock siena half dummi knob with\tschla...,2,use with 1-5/8 in. to 1-3/4 in. door adjustabl...,schlage,...,0.317349,NaN,0.000000,no certifications or listings,0.356998,0.576876,0.152890,The Schlage Camelot In-Active Aged Bronze Hand...,0.072100,0.672977
74065,221471,206648,plastec 11 in. x 24 in. rose garden wall decor...,zen garden decor,3.00,3,zen garden decor\tplastec 11 in. x 24 in. rose...,2,indoor and outdoor use finished in dark bronze...,plastec,...,0.178732,NaN,0.000000,NaN,0.000000,0.367627,0.250415,The Rose Garden is inspired by the popular ear...,0.055013,0.351004


In [18]:
df_complete['brand_match'] = df_complete.apply(
    lambda row: int(str(row['search_term']).lower() in str(row.get('mfg brand name_value', '')).lower()), axis=1
)

def normalized_word_overlap(query, text):
    q_words = set(query.split())
    t_words = set(text.split())
    if not q_words: return 0
    return len(q_words & t_words) / len(q_words)

df_complete['norm_overlap_title'] = df_complete.apply(
    lambda row: normalized_word_overlap(row['search_term'], row['product_title']), axis=1
)

df_complete['length_diff'] = abs(
    df_complete['search_term'].map(lambda x: len(x.split())) - 
    df_complete['product_title'].map(lambda x: len(x.split()))
)

In [19]:
# 1. Get all spacy_sim_* features
spacy_features = [col for col in df_complete.columns if col.startswith('spacy_sim_')]

# 2. Build improved feature set
improved = df_complete[['id', 'relevance', 'cosine_title', "cosine_description", 'len_of_query', 'word_in_title', "brand_match", "norm_overlap_title", "length_diff"] + spacy_features]

# 3. Split using the original indices
train_idx = train.index
test_idx = test.index

X_improved_train = improved.loc[train_idx].drop(['id', 'relevance'], axis=1)
X_improved_test = improved.loc[test_idx].drop(['id', 'relevance'], axis=1)
y_improved_train = improved.loc[train_idx]['relevance'].values
y_improved_test = improved.loc[test_idx]['relevance'].values

# 4. Train improved model
rf_improved = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=42)
clf_improved = BaggingRegressor(rf_improved, n_estimators=45, max_samples=0.1, random_state=42)
clf_improved.fit(X_improved_train, y_improved_train)

# 5. Evaluate
y_improved_pred = clf_improved.predict(X_improved_test)
rmse_improved = np.sqrt(mean_squared_error(y_improved_test, y_improved_pred))

print(f"RMSE with improved features: {rmse_improved:.4f}")
print(f"Baseline RMSE: {rmse:.4f}")


RMSE with improved features: 0.4831
Baseline RMSE: 0.4842


In [20]:
gbr = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
gbr.fit(X_improved_train, y_improved_train)
y_pred_gbr = gbr.predict(X_improved_test)

rmse_gbr = np.sqrt(mean_squared_error(y_improved_test, y_pred_gbr))
print(f"Gradient Boosting RMSE: {rmse_gbr:.4f}")

Gradient Boosting RMSE: 0.4779
